In [4]:
import os
import glob
import argparse
import pandas as pd
from multiprocessing import Pool

parser = argparse.ArgumentParser()
parser.add_argument("--data_dir", "-d", help="The run directory")
parser.add_argument("--chunk_file", "-c", help="The chunk coordiantes of each chromosome")
parser.add_argument("--prephased_g", "-g", help="The prephased result")
parser.add_argument("--genetic_map", "-m", help="The genetic map which comes from HapMap")
parser.add_argument("--reference_hap", "-r", help="The haplotype which comes from 1000 Genome")
parser.add_argument("--reference_legend", "-l", help="The legend which comes from 1000 Genome")
# parser.add_argument("--result_file", "-o", help="The imputed results")
args = parser.parse_args()
#
data_dir = args.data_dir
chunk_file = args.chunk_file
#
prephased_g = args.prephased_g
genetic_map = args.genetic_map
reference_hap = args.reference_hap
reference_legend = args.reference_legend
# result_file = args.result_file
######################
os.chdir(data_dir)
######################

def impute_genotype(start, end):
    prefix = data_dir.strip().split(os.sep)[-1]
    chr_num = os.path.basename(chunk_file).split("_chr")[1].split(".txt")[0]
    result_file = "%s.chr%s_chunk%s-%s.phased.impute2" % (prefix, chr_num, str(start), str(end))
    os.system("/home/software/impute_v2.3.2_x86_64_static/impute2 -use_prephased_g -m %s -h %s -l %s -known_haps_g %s -int %d %d -Ne 20000 -o %s -phase" % (genetic_map, reference_hap, reference_legend, prephased_g, start, end, result_file))
    

df = pd.read_table(chunk_file, header=None)
start_list, end_list = df.iloc[:, 1].tolist(), df.iloc[:, 2].tolist()
assert len(start_list) == len(end_list)
pool = Pool(processes=20)
for i in range(len(start_list)):
    pool.apply_async(impute_genotype, (start_list[i], end_list[i], ))
pool.close()
pool.join()

['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22']
